In [1]:
#importation des packages
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt

In [2]:
#importation des données du baromètre
df=pd.read_csv("C:/Users/sylva/OneDrive/Bureau/barometre/donnees_brutes_2021.csv", sep=";")

C:\Users\sylva\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1,3,68,70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#création d'une variable géographique ville enlevant les problèmes d'homonymie
p=re.compile('(\D*) \(([0-9]{2,2})[0-9]{3,3}\)')
def yo_ville(x):
    m=p.match(x)
    return m.group(1)+' ' +m.group(2)
df['Ville']=df['Commune'].apply(yo_ville)

In [13]:
#la base où sont recensées les intercommunalités
df_inter=pd.read_csv("C:/Users/sylva/OneDrive/Bureau/barometre/intercommunalite.csv", sep=";", skiprows=5)

In [18]:
#on crée une Ville correspondante
df_inter['Ville']=df_inter['LIBGEO']+' '+df_inter['DEP']

In [19]:
#fusion des bases
df_complet=df.merge(df_inter[['Ville', 'LIBEPCI']], how='left')

In [22]:
var_note =['Agencement_voirie', 'conflits_pietons', 'Conducteurs_respectent',
       'Circulation_vehicules_motorises', 'Deplacements_pied_commune',
       'Marche_securite', 'Securite_rues_services',
       'Securite_rues_residentielles', 'Securite_rejoindre_communes_voisines',
       'Acces_pietons', 'Marche_separe_circulation',
       'Marche_separe_autres_mobilites', 'Traverse_carrefour',
       'Deplacements_enfants_personnes_agees', 'Deplacements_enfants_ecole',
       'Pistes_cyclables', 'Espaces_pietons',
       'Espaces_pietons_libres_empietements', 'Espaces_pietons_entretenus',
       'Espaces_pietons_travaux', 'Signaletique_specifique',
       'Amenagements_confort_marcheurs', 'Trottoirs_eclaires',
       'Commune_ecoute_pietons', 'Commune_effort_marche', 'Situation_pietons',
       'Promotion_deplacement', 'Stationnement_espaces_pietons',
       'Acces_transport_commun', 'Signaletique', 'Information']

In [24]:
#on crée des moyennes par intercommunalité
df_note=df_complet[var_note+['Commune']+['Ville']+['LIBEPCI']]
for col in df_note.columns:
    df_note=df_note[df_note[col].notna()]

def dataville_(df, commune='Ville', var_note=var_note) :
    liste_ville=df[commune].unique()
    grosse_ville=[]
    for ville in liste_ville :
        if df[df[commune]==ville][commune].count()>50 :
            grosse_ville+=[ville]#on va se concentrer sur les grandes villes
    dataville=pd.DataFrame()
    dataville[commune]=np.array(grosse_ville)
    for col in var_note:
        liste=[]
        liste_ecart=[]
        for ville in grosse_ville:
            moyenne=0
            nbre_reponse=0
            ecart_type=np.std(df[df[commune]==ville][col])
            for note in df[df[commune]==ville][col]:
                moyenne+=note
                nbre_reponse+=1
            liste.append(moyenne/nbre_reponse)
            liste_ecart.append(ecart_type)
        dataville[col]=np.array(liste)
        dataville[col+'std']=np.array(liste_ecart)
    return dataville
dataville=dataville_(df_note)
datainter=dataville_(df_note, commune='LIBEPCI')

In [26]:
np.mean(dataville['conflits_pietonsstd'])

1.3788800899291496

In [27]:
np.mean(datainter['conflits_pietonsstd'])

1.4036501786202298